In [1]:
import tensorflow as tf
import tensorflow_hub as hub # pip install --upgrade tensorflow-hub
import soundfile as sf
import requests
import numpy as np
from typing import Generator, Iterable, List, Optional
import mediapy as media #3 pip install mediapy
import os
import random
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, ImageSequenceClip, CompositeAudioClip
import subprocess
import ffmpeg
import cv2
from pydub import AudioSegment
import math
import googletrans
from googletrans import Translator

# start high res
""" import argparse
import glob
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from gfpgan import GFPGANer
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
import time
 """



EXTERNAL = False

if EXTERNAL:
  %store -r WIDTH
  %store -r HEIGHT
  %store -r TEXT
  %store -r VISUALIZE
  %store -r SAVE_FOLDER
  %store -r PROJECT_NAME
  %store -r SHOW_OUTPUT
  %store -r IMG_NUMBER
  %store -r THUMBNAIL
  %store -r SKIP_VIDEO
  %store -r MERGE_VIDEO
  %store -r TEXT_VOICE_GEN
  %store -r LANGUAGES
  %store -r GEN_VIDEO
  %store -r INSERT_BACKGROUND
  %store -r RECORD_FRAME_INTER_AFTER
else:
  WIDTH = 768
  HEIGHT = 768
  VISUALIZE = True  
  SAVE_FOLDER = "D:\\Deletar\\p_gen"
  PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
  STYLES_FOLDER = ".\\styles"
  SHOW_OUTPUT = True
  IMG_NUMBER = 2
  SKIP_VIDEO = []
  MERGE_VIDEO = False
  TEXT_VOICE_GEN = ['A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A']
  LANGUAGES = ['pt']
  GEN_VIDEO = True
  INSERT_BACKGROUND = False
  RECORD_FRAME_INTER_AFTER = 0


translator = Translator()

project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"

hub_handle = "https://tfhub.dev/google/film/1"
hub_handle = "./film_1"
model = hub.load(hub_handle)



_UINT8_MAX_F = float(np.iinfo(np.uint8).max)


""" translated_ita = translator.translate('And so, with the money he earned, Jack was able to start his own business and became a successful jeweler himself.', src='en', dest='pt')
print(translated_ita) """

# print(googletrans.LANGUAGES)


" translated_ita = translator.translate('And so, with the money he earned, Jack was able to start his own business and became a successful jeweler himself.', src='en', dest='pt')\nprint(translated_ita) "

In [2]:
""" ANIME = True

path = "./t.jpg"
save_path = "./modified.jpg"

netscale = 4
model_path_face = "./realesrgan/GFPGANv1.3.pth"
dni_weight = None

tile = 0
title_pad = 10
pre_pad = 0
half = None
gpu_id = None
upscale = 3.5
upscale = 4
img_mode = 'RGBA'

if ANIME:
  # R-ESRGAN + Anime
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"
else:
  # R-ESRGAN
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth"

upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path_x4,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
)

face_enhancer = GFPGANer(
            model_path=model_path_face,
            upscale=upscale,
            arch='clean',
            channel_multiplier=2,
            bg_upsampler=upsampler)

def resize_image(data):
  resized_img = tf.image.resize(
  images=data,
  size=[270, 480],
  # size=[180, 320],
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img """

' ANIME = True\n\npath = "./t.jpg"\nsave_path = "./modified.jpg"\n\nnetscale = 4\nmodel_path_face = "./realesrgan/GFPGANv1.3.pth"\ndni_weight = None\n\ntile = 0\ntitle_pad = 10\npre_pad = 0\nhalf = None\ngpu_id = None\nupscale = 3.5\nupscale = 4\nimg_mode = \'RGBA\'\n\nif ANIME:\n  # R-ESRGAN + Anime\n  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)\n  model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"\nelse:\n  # R-ESRGAN\n  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)\n  model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth"\n\nupsampler = RealESRGANer(\n        scale=netscale,\n        model_path=model_path_x4,\n        dni_weight=dni_weight,\n        model=model,\n        tile=tile,\n        tile_pad=title_pad,\n        pre_pad=pre_pad,\n        half= not half,\n        gpu_id= gpu_id\n)\n\nface_enhancer = GFPGANer(\n            model_path=model_path_face,\n            upscale=

In [3]:
durations = []
shared_img = []
set_durations = []
if EXTERNAL:
  for folder_name in os.listdir(os.path.join(f"{project_folder}\\audio\\")):
    for filename in os.listdir(os.path.join(f"{project_folder}\\audio\\{folder_name}")):
      duration = sf.SoundFile(os.path.join(f"{project_folder}\\audio\\{folder_name}\\{filename}"))
      durations.append(duration.frames / duration.samplerate)
  for index, folder_name in enumerate(os.listdir(os.path.join(f"{project_folder}\\style\\"))):
    if len(os.listdir(f"{project_folder}//style//{folder_name}")) > 0:      
      shared_img.append(f"{project_folder}//style//{folder_name}")
    else:
      shared_img.append(shared_img[index-1])
  """ for i, dr in enumerate(durations):
    temp_cond = True
    temp_val = 0
    while (temp_cond):
      for y in range(i, len(durations)):
        if (y == len(durations)):
          if (shared_img[y] == 0):
            set_durations.append(durations[y])
        else:
          if  shared_img[y] == 1:
            temp_val = temp_val + durations[y]
          else:
            set_durations.append(temp_val)
            temp_cond = False
            break """



def _pad_to_align(x, align):

  # Input checking.
  assert np.ndim(x) == 4
  assert align > 0, 'align must be a positive number.'

  height, width = x.shape[-3:-1]
  height_to_pad = (align - height % align) if height % align != 0 else 0
  width_to_pad = (align - width % align) if width % align != 0 else 0

  bbox_to_pad = {
      'offset_height': height_to_pad // 2,
      'offset_width': width_to_pad // 2,
      'target_height': height + height_to_pad,
      'target_width': width + width_to_pad
  }
  padded_x = tf.image.pad_to_bounding_box(x, **bbox_to_pad)
  bbox_to_crop = {
      'offset_height': height_to_pad // 2,
      'offset_width': width_to_pad // 2,
      'target_height': height,
      'target_width': width
  }
  return padded_x, bbox_to_crop

def load_image(img_url: str):  

  if (img_url.startswith("https")):
    user_agent = {'User-agent': 'Colab Sample (https://tensorflow.org)'}
    response = requests.get(img_url, headers=user_agent)
    image_data = response.content
  else:
    image_data = tf.io.read_file(img_url)

  image = tf.io.decode_image(image_data, channels=3)
  image_numpy = tf.cast(image, dtype=tf.float32) #.numpy()
  image_numpy = tf.image.resize(
    images=image_numpy,
    # size=[HEIGHT, WIDTH],
    size=[270, 480],
    method=tf.image.ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=True,
    ).numpy()
  """ image_numpy = np.clip(image_numpy, 0, 255)  """
  return image_numpy / _UINT8_MAX_F

class Interpolator:

  def __init__(self, align: int = 64) -> None:

    #self._model = hub.load("https://tfhub.dev/google/film/1")
    self._model = hub.load(".\\film_1")
    self._align = align

  def __call__(self, x0: np.ndarray, x1: np.ndarray,
               dt: np.ndarray) -> np.ndarray:

    if self._align is not None:
      x0, bbox_to_crop = _pad_to_align(x0, self._align)
      x1, _ = _pad_to_align(x1, self._align)

    inputs = {'x0': x0, 'x1': x1, 'time': dt[..., np.newaxis]}
    result = self._model(inputs, training=False)
    image = result['image']

    if self._align is not None:
      image = tf.image.crop_to_bounding_box(image, **bbox_to_crop)
    return image.numpy()

def _recursive_generator(
    frame1: np.ndarray, frame2: np.ndarray, num_recursions: int,
    interpolator: Interpolator) -> Generator[np.ndarray, None, None]:

  if num_recursions == 0:
    yield frame1
  else:
    # Adds the batch dimension to all inputs before calling the interpolator,
    # and remove it afterwards.
    time = np.full(shape=(1,), fill_value=0.5, dtype=np.float32)
    mid_frame = interpolator(
        np.expand_dims(frame1, axis=0), np.expand_dims(frame2, axis=0), time)[0]
    yield from _recursive_generator(frame1, mid_frame, num_recursions - 1,
                                    interpolator)
    yield from _recursive_generator(mid_frame, frame2, num_recursions - 1,
                                    interpolator)

def interpolate_recursively(
    frames: List[np.ndarray], num_recursions: int,
    interpolator: Interpolator) -> Iterable[np.ndarray]:

  n = len(frames)
  for i in range(1, n):
    yield from _recursive_generator(frames[i - 1], frames[i],
                                    times_to_interpolate, interpolator)
  # Separately yield the final frame.
  yield frames[-1]



def get_all_files(folder):
  temp_array = []
  for filename in os.listdir(folder):
      temp_array.append(f"{folder}//{filename}")
  return temp_array


def concat_videos(videos, output_path):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = 0
    frame_height = 0
    frame_count = 0
    audio_segments = []
    video_list = []

    for video in videos:
        cap = cv2.VideoCapture(video)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        frame_count += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        video_list.append(cap)
        audio = AudioSegment.from_file(video, "mp4")
        audio_segments.append(audio)

    out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))
    frame_index = 0

    while True:
        finished = True
        for cap in video_list:
            if cap.isOpened():
                finished = False
                ret, frame = cap.read()
                if ret == True:
                    out.write(frame)
                    frame_index += 1
                else:
                    cap.release()

        if finished or frame_index >= frame_count:
            break

    out.release()
    cv2.destroyAllWindows()

    final_audio = AudioSegment.silent(duration=0)
    for audio in audio_segments:
        final_audio += audio

    final_audio.export(output_path.replace(".mp4", ".wav"), format="wav")

""" def high_res_img(img):

  img = np.clip(img, 0, 255)  
  _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
  return output """
  


#create srt
def create_srt(text, initial_time, final_time, indx, filename):
  initial_mil = int(round(initial_time % 1,3)*1000)
  final_mil = int(round(final_time % 1,3)*1000)
  initial_time = int(initial_time)
  final_time = int(final_time)

  minutes, seconds = divmod(initial_time, 60)
  hours, minutes = divmod(minutes, 60)
  initial_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{initial_mil}'

  minutes, seconds = divmod(final_time, 60)
  hours, minutes = divmod(minutes, 60)
  final_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{final_mil}'

  with open(f"{filename}//en.srt" , 'w', encoding='utf-8') as f:
    f.write(f'{int(indx)+1}\n')
    f.write(f'{initial_time} --> {final_time}\n')
    f.write(f'{text}\n')
    f.write('\n')
  
  for lang in LANGUAGES:
    transl = translator.translate(text, src='en', dest=lang).text
    with open(f"{filename}//{lang}.srt" , 'w', encoding='utf-8') as f:
      f.write(f'{int(indx)+1}\n')
      f.write(f'{initial_time} --> {final_time}\n')
      f.write(f'{transl}\n')
      f.write('\n')
  

def merge_srt_files(filenames, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for i, filename in enumerate(filenames):
            with open(filename, 'r', encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line)
                if i < len(filenames) - 1:
                    outfile.write('\n')

""" create_srt('Once upon a time, in a small town nestled in the heart of the countryside, there lived a young boy named Jack.', 0.5, 7.62, 0, filename=f"{project_folder}//video_sound//{0:07d}") """

times_to_interpolate = 1
interpolator = Interpolator()

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            img_url = os.path.join(folder_path, filename)
            image_data = tf.io.read_file(img_url)
            image = tf.io.decode_image(image_data, channels=3)
            image_numpy = tf.cast(image, dtype=tf.float32) #.numpy()
            image_numpy = tf.image.resize(
                images=image_numpy,
                size=[270, 480],
                method=tf.image.ResizeMethod.BILINEAR,
                preserve_aspect_ratio=False,
                antialias=True,
            ).numpy()
            images.append(image_numpy / _UINT8_MAX_F)
    return images

netscale = 4
model_path_face = "./realesrgan/GFPGANv1.3.pth"
dni_weight = None
tile = 0
title_pad = 10
pre_pad = 0
half = None
gpu_id = None
upscale = 4
img_mode = 'RGBA'

print(f"Face Enhance = {FACE_ENHANCE}")

if ANIME:  
    if not SM_MODEL:
      print('loading anime 4x')
      model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
      model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"
      """ else:
      # R-ESRGAN
      model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
      model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth" """

      upsampler = RealESRGANer(
              scale=netscale,
              model_path=model_path_x4,
              dni_weight=dni_weight,
              model=model,
              tile=tile,
              tile_pad=title_pad,
              pre_pad=pre_pad,
              half= not half,
              gpu_id= gpu_id
      )

      face_enhancer = GFPGANer(
                  model_path=model_path_face,
                  upscale=upscale,
                  arch='clean',
                  channel_multiplier=2,
                  bg_upsampler=upsampler)
    else:
      print('loading anime 2x')
      model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=16, upscale=4, act_type='prelu')
      model_path_x2 = "./realesrgan/realesr-animevideov3.pth"
      upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path_x2,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
      )

      """ from gfpgan import GFPGANer """
      face_enhancer = GFPGANer(
                  model_path=model_path_face,
                  upscale=upscale,
                  arch='clean',
                  channel_multiplier=2,
                  bg_upsampler=upsampler)
        
else:
  if not SM_MODEL:
    print('loading real HD')
    from basicsr.archs.rrdbnet_arch import RRDBNet
    arch = 'clean'
    channel_multiplier = 2
    model_name = 'GFPGANv1.4'
    url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'
    model_path = os.path.join('./gfpgan/gfpgan/weights', model_name + '.pth')
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
    upsampler = RealESRGANer(
              scale=2,
              model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
              model=model,
              tile=400,
              tile_pad=10,
              pre_pad=0,
              half=True)  # need to set False in CPU mode

    face_enhancer = GFPGANer(
      model_path=model_path,
      upscale=upscale,
      arch=arch,
      channel_multiplier=channel_multiplier,
      bg_upsampler=upsampler)

  else: 
    print('loading real SM')
    model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
    file_url = [
            'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-wdn-x4v3.pth',
            'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth'
    ]
    model_path = os.path.join('weights', 'realesr-general-x4v3' + '.pth')
    if not os.path.isfile(model_path):
        ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
        for url in file_url:
            # model_path will be updated
            model_path = load_file_from_url(
                url=url, model_dir=os.path.join(ROOT_DIR, 'weights'), progress=True, file_name=None)
  
    wdn_model_path = model_path.replace('realesr-general-x4v3', 'realesr-general-wdn-x4v3')
    model_path = [model_path, wdn_model_path]
    dni_weight = [0.5, 1 - 0.5]
    
    upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
    )


    from gfpgan import GFPGANer
    face_enhancer = GFPGANer(
        model_path=model_path_face,
        upscale=upscale,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=upsampler)

In [4]:
""" 
    each photo add 2.7s of total video
    2.7 - 2 fotos - 18s
    5.375 - 3 fotos - 36 s
    8 - 4 fotos - 53.7s
    10.7 - 5 fotos - 71s 
"""

""" EXTERNAL = False """


if EXTERNAL:
    initial_img = ""
    final_img = ""
    used_img = []
    start_time = 0
    
    for index, folder_name in enumerate(shared_img):
        img_array = []        
        num_of_pictures = int(math.ceil(durations[index] / 2.7) + 1)
        files_inside_folder = []
        files_inside_folder = get_all_files(folder_name)
        files_inside_folder = sorted(files_inside_folder)
        
        if index != len(shared_img)-1:
            files_inside_next_folder = get_all_files(shared_img[index+1])
            files_inside_next_folder = [i for i in files_inside_next_folder if i not in used_img]
            files_inside_next_folder = sorted(files_inside_next_folder)   
            if index == 0:
                if (files_inside_folder == files_inside_next_folder):
                    rdn_number = random.randint(1, len(files_inside_next_folder)-1)  
                    temp = files_inside_next_folder[rdn_number]         
                    used_img.append(temp)
                else:
                    temp = files_inside_next_folder[0]
                    used_img.append(temp)                    
            elif (files_inside_next_folder[0][-5:] == "c.jpg" and files_inside_next_folder[0] not in used_img):
                temp = files_inside_next_folder[0]
                used_img.append(temp)
            else:
                rdn_number = random.randint(0, len(files_inside_next_folder)-1)  
                temp = files_inside_next_folder[rdn_number]         
                used_img.append(temp)
            num_of_pictures = num_of_pictures - 1
            final_img = temp           
        else:
            final_img = ''
        
        
        
        files_inside_folder = [i for i in files_inside_folder if i not in used_img]

        
        if initial_img != "":
            img_array.append(initial_img)
            num_of_pictures = num_of_pictures - 1
        

        if (num_of_pictures > 0):
            if (index == 0):
                num_of_pictures = num_of_pictures - 1
                rd = random.sample(range(1, len(files_inside_folder)), num_of_pictures)
            else:
                rd = random.sample(range(0, len(files_inside_folder)), num_of_pictures)
            for i in rd:
                img_array.append(files_inside_folder[i])
        

        if final_img != "":
            img_array.append(final_img)            
            initial_img = final_img

        if (index == 0):
            img_array.insert(0,files_inside_folder[0])            
            

        
        

        for idx, i in enumerate(img_array):
           used_img.append(img_array[idx])
        
        """ if index <= RECORD_FRAME_INTER_AFTER:
            if os.path.exists(f"{project_folder}//video_sound//{index:07d}//{index:05d}.mp4"):
                get_video_dur = VideoFileClip(f"{project_folder}//video_sound//{index:07d}//{index:05d}.mp4")
                start_time = start_time + round(get_video_dur.duration,3) """
            


        if index not in SKIP_VIDEO and GEN_VIDEO and index > RECORD_FRAME_INTER_AFTER:
            input_frames = []
            """ if (not os.path.exists(f"{project_folder}//video")):
                os.makedirs(f"{project_folder}//video")
            if (not os.path.exists(f"{project_folder}//video//{int(index):07d}")):
                os.makedirs(f"{project_folder}//video//{int(index):07d}") """
            for i in img_array:
                input_frames.append(load_image(i))
                

            frames = list(
                interpolate_recursively(input_frames, times_to_interpolate,
                interpolator))
            """ print(f'video with {len(frames)} frames')
            media.show_video(frames, fps=24, title='FILM interpolated video')
            media.write_video(f"{project_folder}//video//{index:07d}//{index:05d}.mp4",frames, fps=24)
            new_video =
            VideoFileClip(f"{project_folder}//video//{index:07d}//{index:05d}.mp4")
            """
            # frames = [np.rint(np.array(img)*255) for img in frames]
            # frames = [np.array(img)*255 for img in frames]
            
            """ for indx, img in enumerate(frames):
                frames[indx] = high_res_img(img*255) """

            """ frames = [np.array(high_res_img((img*255))) for img in frames] """
            """ frames = [high_res_img(np.array(img)*255) for img in frames] """
            ('print after frames')
            
            
            """ frames = [np.array(img)*255 for img in frames]    """        
            frames = [np.clip(np.array(img)*255,0, 255) for img in frames]
            
            video = ImageSequenceClip(frames, fps=30)
            """ video.write_videofile(f"{project_folder}//video//{index:07d}//{index:05d}.mp4", codec='libx264') """
            audio = AudioFileClip(f"{project_folder}//audio//{index:07d}//{index:05d}.wav")
            audio = audio.set_end(audio.duration -0.1)
            
            final_video = video.set_audio(audio)            
            """ audio_duration = round(audio.duration,3)
            max_duration_clip = round(max(audio.duration,final_video.duration),3) """
            
            if (not os.path.exists(f"{project_folder}//video_sound")):
                os.makedirs(f"{project_folder}//video_sound")
            if (not os.path.exists(f"{project_folder}//video_sound//{int(index):07d}")):
                os.makedirs(f"{project_folder}//video_sound//{int(index):07d}")
            if os.path.exists(f"{project_folder}//video_sound//{index:07d}//{index:05d}.mp4"):
                os.remove(f"{project_folder}//video_sound//{index:07d}//{index:05d}.mp4")
            final_video.write_videofile(f"{project_folder}//video_sound//{index:07d}//{index:05d}.mp4")
            """ create_srt(TEXT_VOICE_GEN[str(index)], start_time + 0.5, start_time + audio_duration, index, filename=f"{project_folder}//video_sound//{index:07d}")
            start_time = round(start_time + max_duration_clip,3) """


    """ if (not os.path.exists(f"{project_folder}//full")):
                os.makedirs(f"{project_folder}//full")
    frames = list(
                interpolate_recursively(whole_video, times_to_interpolate,
                interpolator))
    media.write_video(f"{project_folder}//full//full.mp4",frames, fps=24) """
else:
    dir_location = "F:\\gg\\templates\\a globe inside a jar\\1"
    input_frames = load_images_from_folder(dir_location)
    
    """     image_1_url = ".\\result5.jpg"
    image_2_url = ".\\result3.jpg"
    image_3_url = ".\\result4.jpg"
    image_4_url = ".\\result_a3.png"    
    image_5_url = ".\\result_a4.png"
    image_6_url = "result_a5.png" """
    """image_7_url = "D:\\OneDrive\\Desktop\\machine-2\\result_a5.jpg"
    image_8_url = "D:\\OneDrive\\Desktop\\machine-2\\result_a6.jpg"
    image_8_url = "D:\\OneDrive\\Desktop\\machine-2\\result_a7.jpg"
    image_9_url = "D:\\OneDrive\\Desktop\\machine-2\\result_a8.jpg" """

    """     image1 = load_image(image_1_url)
    image2 = load_image(image_2_url)
    image3 = load_image(image_3_url) """
    """ image4 = load_image(image_4_url) """
    
    
    
    """ image5 = load_image(image_5_url)
    image6 = load_image(image_6_url) """
    """ image7 = load_image(image_7_url)
    image8 = load_image(image_8_url)
    image9 = load_image(image_9_url) """


    """ input_frames = [image1, image2, image3, image4, image5, image6, image7,
    image8, image9] """
    """ input_frames = [image1,image2, image3] """
    # input_frames = [image1,image2,image3,image4,image5]

    frames = list(
        interpolate_recursively(input_frames, times_to_interpolate,
                                            interpolator))
    print(f'video with {len(frames)} frames')

    """ for index, img in enumerate(frames):
        frames[index] = high_res_img(img*255) """

    media.show_video(frames, fps=30, title='FILM interpolated video')



video with 477 frames


FILM interpolated video This browser does not support the video tag.


In [5]:

""" MERGE_VIDEO = True """
# merged_arr = []
if MERGE_VIDEO and EXTERNAL:
    video = []
    """ filenames = []
    filenames_other = {}
    for lan in LANGUAGES:
        filenames_other[lan] = [] """
    
    if (not os.path.exists(f"{project_folder}//full")):
                os.makedirs(f"{project_folder}//full")
    for folder in os.listdir(f"{project_folder}//video_sound"):
        for filename in os.listdir(f"{project_folder}//video_sound//{folder}"):
            if (filename[-3:] == 'mp4'):
                addr_video = f"{project_folder}//video_sound//{folder}//{filename}"
                # merged_arr.append(addr_video)
                video_clip = VideoFileClip(addr_video)                     
                video.append(video_clip)
            """ elif (filename[-6:] == 'en.srt'):
                filenames.append(f"{project_folder}//video_sound//{folder}//{filename}")
            elif(filename[-3:] == 'srt'):
                lan = filename[-6:-4]
                filenames_other[lan] = filenames_other[lan] + [f"{project_folder}//video_sound//{folder}//{filename}"] """

    
    merged_video = concatenate_videoclips(video, method='chain')
    if INSERT_BACKGROUND == True:
        original_audio = merged_video.audio
        music  = AudioFileClip("./rain.wav")
        music = music.volumex(0.1)
        music = music.audio_loop(duration = merged_video.duration)
        combined_audio = CompositeAudioClip([original_audio, music])
        merged_video = merged_video.set_audio(combined_audio)
    if os.path.exists(f"{project_folder}//full//full.mp4"):
        os.remove(f"{project_folder}//full//full.mp4")
    merged_video.write_videofile(f"{project_folder}//full//full.mp4")
    """ if len(filenames) > 0:
        merge_srt_files(filenames, f"{project_folder}//full//full.srt")
        for lang_mod in filenames_other:
            merge_srt_files(filenames_other[lang_mod], f"{project_folder}//full//{lang_mod}.srt") """

    """ concat_videos(merged_arr,f"{project_folder}//full//full.mp4") """


    """ input_streams = [ffmpeg.input(file) for file in addr_video]
    ffmpeg.concat(*input_streams, v=1, a=1).output("output.mp4", c="copy").run() """
   